<h1 style="font-family: 'Georgia', serif; text-align: center; font-size: 30px; color:rgb(52, 152, 219); text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3)">Modélisation</h1>
<br/>
<br/>
  

## Importation des données

In [1]:
import os
#librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

current_directory = os.getcwd()
new_directory = current_directory.replace("src", "data/HousingTreated.csv")
df= pd.read_csv(new_directory)
# Imputer les colonnes contenant des NaN avec leur moyenne
df = df.fillna(df.mean())
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   NOX      506 non-null    float64
 4   RM       506 non-null    float64
 5   AGE      506 non-null    float64
 6   DIS      506 non-null    float64
 7   RAD      506 non-null    float64
 8   TAX      506 non-null    int64  
 9   PTRATIO  506 non-null    float64
 10  B        506 non-null    float64
 11  LSTAT    506 non-null    float64
 12  MEDV     506 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 51.5 KB


In [12]:
#separation des var

X = df.drop('MEDV', axis =1)
Y=df['MEDV']

#var d'entrainnement
seed= 111
test_size = 0.2
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size= test_size, random_state=seed)
print("X_train :", X_train.shape)
print("Y_train :", Y_train.shape)
print("X_test :", X_test.shape)
print("Y_test :", Y_test.shape)


X_train : (404, 12)
Y_train : (404,)
X_test : (102, 12)
Y_test : (102,)


<p>
Nous avons divisé les données en données d'entrainnement et données d'évaluation pour verifier si le modèle prédit bien. les données d'entrainnement représentent 20% des données(test_size).
Passons désormais à la construction du modèle
</p>

In [3]:
model = LinearRegression()
model.fit(X_train,Y_train)


LinearRegression()

<p>
Nous allons à présent calculer certains paramètres clés qui permettrons de juger le modèle sur les données d'entrainnement
</p>

In [4]:
#coef de determination
print("le coefficiant de determination ou score est :",model.score(X_train,Y_train))
#coefficiants du modèle
print("les coefficiants du modele sont :",model.coef_)
#prediction sur les données d'entrainnement
Y_pred = model.predict(X_train)
#erreur quadratique
print("l'erreur quadratique est :",mean_squared_error(Y_train,Y_pred))


le coefficiant de determination ou score est : 0.7599097743074853
les coefficiants du modele sont : [-7.37938409e-03 -9.71324364e-05  4.19401648e-04 -6.89000318e-01
  1.26312163e-01 -1.00762313e-03 -2.13330753e-01  8.73218225e-02
 -5.72821081e-04 -3.44650406e-02  6.17027816e-04 -2.81385156e-02]
l'erreur quadratique est : 0.03970892123982534


Nous faisons de meme pour les données de test

In [5]:
#coef de determination
print("le coefficiant de determination ou R2 est :",model.score(X_test,Y_test))

#prediction sur les données d'entrainnement
Y_hat = model.predict(X_test)
#erreur quadratique
print("l'erreur quadratique est :",mean_squared_error(Y_test,Y_hat))


le coefficiant de determination ou R2 est : 0.7057505025041648
l'erreur quadratique est : 0.05046075311883629


<p>
nous constatons que le R<sup>2</sup> des données test(0,705) est inférieur à celui des données d'entrainnement(0,759) 
l'erreur quadratique sur les données de test(0,05) est un peu plus élevé que sur les données d'entrainnement(0,04)<br/>
Nous pouvons donc conclure qu'il n'ya pas de Problème de réajustement et notre modèle est capable de bien se généraliser
</p>

<br/>
<h1 style="font-family: 'Georgia', serif; text-align: center; font-size: 30px; color:rgb(52, 152, 219); text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3)">validation Croisée du Modèle </h1>
<br/>
<br/>
  

<p>
Bien que le modèle est capable de bien s'ajuster à de nouvelles données, les données de test peuvent avoir certaines particularités qui font que le R<sup>2</sup> ait donné telle ou telle valeur. Pour palier à ce problème, due essentiellement à la division arbitraitre de la fonction train_test_split(), nous utilisons la validation croisée
</p>

Pour ce faire, nous allons : 
<ul style="list-style-type: none;">
<li>➤ Diviser le DataFrame en 5 groupes ou plis</li>
<li>➤ Ensuite, le 1er pli est pris comme ensemble de test et l'algorithme est entrainé avec les 4 plis restant. on fait les prédictions sur l'ensemble de test, puis on calcule la métrique d'évaluation du modèle</li>
<li>➤ on fais de meme jusqu'à ce que chacun des 5 plis ait été utilisé comme ensemble de test</li>
<p>
on aura donc finalement 5 modèle avec 5 métriques (R<sup>2</sup> par exemple) et on peut calculer des statistiques comme la moyenne ou la médiane afin de voir leurs repartitions. 
</p>
</ul>
<br/>

In [7]:
#creation d'un modèle
linear_model = LinearRegression()
#validation croisée avec 5 plis

cv_scores= cross_val_score(linear_model,X,Y, cv=5)
print("les R carré des différents cas sont",cv_scores)

les R carré des différents cas sont [0.69197213 0.76253904 0.60858392 0.36941686 0.40734423]


<p>
Nous avons 5 R<sup>2</sup> et nous pouvons donc calculer la moyenne et la médiane
<p>

In [8]:
mean =np.mean(cv_scores)
median = np.median(cv_scores)
print("la moyenne est :",mean,"et la mediane vaut :",median)

la moyenne est : 0.5679712353288597 et la mediane vaut : 0.6085839157867676


le R<sup>2</sup> médian vaut 0,608. bien que cette valeur est plus petite que (0,705) celle obtenue en faisant train_test_split() , nous pouvons valider le modèle et etre plus confiant sur sa qualité. Ainsi, plus de 60% de la variabilité de la valeur médiane des logements est expliquée par les variables de notre modèle

In [ ]:
import statsmodels.api as sm

# Ajustement du modèle (vous avez déjà `lm.fit(X, y)`)
# On doit rajouter une constante pour les statistiques
X_with_const = sm.add_constant(X)  # Ajoute une colonne de 1 pour l'intercept

# Utilisation de statsmodels pour calculer les p-values
model = sm.OLS(Y, X_with_const).fit()
#Modele final
lm=LinearRegression()
lm.fit(X,Y)
#coefficiants
print("les coefficiants de la regression sont",lm.coef_)
#ordonnée à l'origine
print("la constante de la regression vaut",lm.intercept_)
print("\nLes p-values correspondantes :",model.pvalues)

les coefficiants de la regression sont [-1.79433288e-02 -2.11692864e-03 -1.85522824e-04 -7.11594043e-01
  1.19210400e-01 -4.41052352e-04 -2.16156292e-01  8.79220719e-02
 -5.36135485e-04 -3.65557540e-02  4.77698728e-04 -2.86036604e-02]
la constante de la regression vaut 3.8794760495005924

Les p-values correspondantes :

Les p-values correspondantes : const      5.894814e-51
CRIM       8.817607e-02
ZN         8.057837e-01
INDUS      9.425998e-01
NOX        4.553124e-05
RM         3.171469e-11
AGE        4.329838e-01
DIS        1.216033e-08
RAD        2.201109e-04
TAX        4.690983e-05
PTRATIO    5.128436e-10
B          4.958294e-05
LSTAT      6.152412e-35
dtype: float64


Nous remarquons aussi que tous les coeffiçiants sont statistiquement très significatifs(significatifs à 1%)

<p>Finalement l'équation du modèle s'écrit : 
<br/>
<b> log(MEDV) = 3,88 - 1,79*10<SUP>-2</sup>log(CRIM) -2,12*10<SUP>-3</sup>log(ZN+1) -1,86*10<SUP>-4</sup>INDUS
<br/>  -7,12*10<SUP>-1</sup>NOS + 1,19*10<SUP>-1</sup>RM - 4,41*10<SUP>-4</sup>AGE
<br/>   -2,16*10<SUP>-1</sup>log(DIS) + 8,79*10<SUP>-2</sup>log(RAD) - 5,36*10<SUP>-4</sup>TAX - 3,66*10<SUP>-2</sup>PTRATIO 
<br/>  + 4,78*10<SUP>-4</sup>B -2,86*10<SUP>-2</sup>LSTAT
</b>
</p>
<br/>

<h1>Effet de chaque variable (en gardant les coefficients significatifs) :</h1>
<ul>
<li>CRIM (-1,79 × 10⁻²) : Une augmentation de 1 % de CRIM entraîne une diminution de 1,79 % de MEDV (puisque la variable est en logarithme).
<li>ZN (-2,12 × 10⁻³) : Une augmentation de 1 % de ZN+1 entraîne une légère baisse du logarithme de MEDV de 0,212 %.
<li>INDUS (-1,86 × 10⁻⁴) : Une augmentation de 1 unité d'INDUS entraîne une légère baisse de MEDV.
<li>RM (+1,19 × 10⁻¹) : Une augmentation d'une unité de RM (nombre moyen de pièces) augmente log(MEDV) de 11,9 %.
<li>AGE (-4,41 × 10⁻⁴) : Une augmentation d'une unité de l'âge des habitations entraîne une baisse légère.
<li>DIS (-2,16 × 10⁻¹) : Une augmentation de 1 % de la distance moyenne entraîne une baisse significative de MEDV de 21,6 %.
<li>RAD (+8,79 × 10⁻²) : Une augmentation logarithmique du RAD (proximité autoroutière) augmente légèrement MEDV.
<li>TAX (-5,36 × 10⁻⁴) : Une hausse d'une unité dans les taxes entraîne une faible diminution de MEDV.
<li>PTRATIO (-3,66 × 10⁻²) : Une augmentation de 1 point du ratio élèves/professeurs entraîne une diminution de 3,66 % de MEDV.
<li>B (+4,78 × 10⁻⁴) : Une augmentation de la proportion de population noire semble légèrement positive.
<li>LSTAT (-2,86 × 10⁻²) : Une augmentation de LSTAT (pourcentage de classe défavorisée) réduit log(MEDV) de 2,86 %.
</ul>
<h1><u>Synthèse </u>:</h1>
<p>Les variables RM, PTRATIO, CRIM, LSTAT et DIS semblent les plus influentes.
RM (nombre de pièces) a un effet très positif, tandis que les facteurs comme le crime (CRIM), la proximité autoroutière excessive (DIS), et la classe sociale défavorisée (LSTAT) réduisent significativement la valeur des propriétés.
</p>

<u><b>Rappel :</b></u> 
<p>
nous avions précédemment remplacés certaines valeurs par leur logarithme afin de satisfaire des hypothèses nécessaires pour la regression. Pour avoir la vriae valeur de MEDV il suffit d'appliquer la fonction numpy.exp()
</p>

<br/>
<h1 style="font-family: 'Georgia', serif; text-align: center; font-size: 30px; color:rgb(52, 152, 219); text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3)">CONCLUSION </h1>
<br/>
<br/>
  

<p>
Au cours de ce projet, nous avons entrepris une analyse approfondie des données afin de modéliser la relation entre plusieurs variables explicatives et la variable cible, MEDV (prix médian des logements).

<b>Étapes clés réalisées </b>:
➤Analyse exploratoire des données :
Nous avons examiné la distribution des différentes variables, mis en évidence les corrélations et identifié les facteurs les plus influents. Cette phase a permis d'orienter les choix méthodologiques pour le modèle.

➤Traitement des données :
Une variable présentant une asymétrie élevée a été retirée afin de garantir la stabilité et la robustesse du modèle. Cette décision était justifiée par une meilleure normalité des données après cette suppression.

➤Construction du modèle de régression linéaire :
Nous avons construit un modèle de régression linéaire classique après avoir ajusté les données en prenant des logarithmes sur certaines variables pour atténuer les effets des outliers et des non-linéarités.

➤Validation croisée :
Une validation croisée a été réalisée pour évaluer la généralisation du modèle. Cela nous a permis de confirmer la cohérence et la fiabilité des coefficients estimés.

<b>Résultats finaux </b>:
Le modèle final obtenu présente une bonne qualité de prédiction avec des coefficients cohérents. Les p-values associées à certains coefficients indiquent que certaines variables ont une influence statistiquement significative sur la valeur des logements, notamment le nombre moyen de pièces (RM), le taux de criminalité (CRIM) et le pourcentage de classe défavorisée (LSTAT).

Finalement, le modèle obtenu fournit une bonne compréhension des facteurs socio-économiques et environnementaux influençant les prix des logements. Cette analyse pourrait servir de base à des études complémentaires, notamment par l'intégration de modèles plus complexes (comme la régression régulière ou les méthodes non-linéaires)
</p>